In [1]:
import pandas as pd
import tabula 
from urllib import request
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urljoin
import re
from PyPDF2 import PdfFileReader
import matplotlib.pyplot as plt
import yfinance as yf

In [2]:
url = "https://petrobras.com.br/pt/nossas-atividades/precos-de-venda-as-distribuidoras/gasolina-e-diesel/"
html = urlopen(url)
bs = BeautifulSoup(html, "html.parser")

# DIESEL
links = []
for x in bs.find_all("a", href = True):
    if re.search(re.compile("diesel", flags = re.IGNORECASE), str(x)):
        links.append(x["href"])

In [3]:
root = "https://petrobras.com.br/"
link_final_diesel = links[2][9:]
link_diesel = root + link_final_diesel

In [4]:
pag_pdf = tabula.read_pdf(link_diesel, pages = "all", multiple_tables=True)
pag_pdf

[                    0                     1           2           3   \
 0                LOCAL  MODALIDADE\rDE VENDA  01.08.2019  05.09.2019   
 1       Araucária (PR)                   EXA    2.062,10    2.114,60   
 2       Araucária (PR)                   LPA    2.064,10    2.116,60   
 3         Barueri (SP)                   EXA    2.088,90    2.141,40   
 4         Barueri (SP)                   LPA    2.093,20    2.145,70   
 ..                 ...                   ...         ...         ...   
 72     Vila Velha (ES)                   LTM    2.131,40    2.183,90   
 73        Vitória (ES)                   EXA    2.186,40    2.238,90   
 74        Vitória (ES)                   LTM    2.131,40    2.183,90   
 75  Volta Redonda (RJ)                   EXA    2.115,40    2.167,90   
 76  Volta Redonda (RJ)                   LPC    2.126,40    2.178,90   
 
             4           5           6           7           8           9   \
 0   13.09.2019  19.09.2019  25.10.2019  01

In [30]:
agrupado = pd.DataFrame()
for dataframe in range(0,len(pag_pdf)):
    
    if len(pag_pdf[dataframe]) > 25:        
        df = pag_pdf[dataframe]
        
        for line_num, line in enumerate(pag_pdf[dataframe].iloc[:,0]):
            if line == "LOCAL":
                linha_datas = df.iloc[line_num, :]
        
        df2 = df.dropna(subset = [df.columns[0]])
#         print(linha_datas)
#         print(df2.head(20))
        try:
            df2.columns = linha_datas
        except:
            print(f"erro página {dataframe}")
            continue
        df2 = df2.reset_index(drop = True)
        df2.drop(0, axis = 0, inplace = True)
        df3 = pd.melt(df2, id_vars = df2.columns[:2], value_vars = df2.columns[2:])
        df3.columns = ["LOCAL", "MODALIDADE", "DATA", "VALOR"]
        agrupado = pd.concat([agrupado, df3], axis = 0)

erro página 10


In [31]:
print(agrupado.to_string())

                            LOCAL MODALIDADE        DATA     VALOR
0                  Araucária (PR)        EXA  01.08.2019  2.062,10
1                  Araucária (PR)        LPA  01.08.2019  2.064,10
2                    Barueri (SP)        EXA  01.08.2019  2.088,90
3                    Barueri (SP)        LPA  01.08.2019  2.093,20
4                      Belém (PA)        ETM  01.08.2019       NaN
5                      Belém (PA)        EXA  01.08.2019  2.058,70
6                      Belém (PA)        LTM  01.08.2019  2.003,70
7                      Betim (MG)        EXA  01.08.2019  2.157,30
8                      Betim (MG)        LPA  01.08.2019  2.159,30
9                    Biguaçu (SC)        EXA  01.08.2019  2.113,20
10                   Biguaçu (SC)        LCT  01.08.2019  2.129,90
11                  Brasília (DF)        EXA  01.08.2019  2.202,50
12                  Brasília (DF)        LPA  01.08.2019  2.207,70
13                  Cabedelo (PB)        EXA  01.08.2019  2.07

In [32]:
agrupado.dropna(subset = ["VALOR"], inplace=True)

agrupado.loc[:,"VALOR"] = agrupado["VALOR"].str.replace(".", "")
agrupado.loc[:,"VALOR"] = agrupado["VALOR"].apply(lambda x: float(x.replace(",", "."))/1000)
agrupado.loc[:,"DATA"] = agrupado["DATA"].apply(lambda x: pd.to_datetime(x, format = "%d.%m.%Y"))

<ipython-input-32-c1e379ac5244>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  agrupado.loc[:,"VALOR"] = agrupado["VALOR"].str.replace(".", "")


In [25]:
#agrupado["last_date"] = agrupado["DATA"].apply(lambda x: 1 if x == agrupado["DATA"].max() else 0)

In [33]:
agrupado

,LOCAL,MODALIDADE,DATA,VALOR
0,Araucária (PR),EXA,2019-08-01,2.0621
1,Araucária (PR),LPA,2019-08-01,2.0641
2,Barueri (SP),EXA,2019-08-01,2.0889
3,Barueri (SP),LPA,2019-08-01,2.0932
5,Belém (PA),EXA,2019-08-01,2.0587
...,...,...,...,...
265,Uberaba (MG),LCT,2021-08-01,2.9402
266,Uberlândia (MG),EXA,2021-08-01,2.9396
267,Uberlândia (MG),LCT,2021-08-01,2.9513
269,Vila Velha (ES),LTM,2021-08-01,2.8101


In [35]:
agrupado

,LOCAL,MODALIDADE,DATA,VALOR,PRODUTO
0,Araucária (PR),EXA,2019-08-01,2.0621,0
1,Araucária (PR),LPA,2019-08-01,2.0641,0
2,Barueri (SP),EXA,2019-08-01,2.0889,0
3,Barueri (SP),LPA,2019-08-01,2.0932,0
5,Belém (PA),EXA,2019-08-01,2.0587,0
...,...,...,...,...,...
265,Uberaba (MG),LCT,2021-08-01,2.9402,0
266,Uberlândia (MG),EXA,2021-08-01,2.9396,0
267,Uberlândia (MG),LCT,2021-08-01,2.9513,0
269,Vila Velha (ES),LTM,2021-08-01,2.8101,0


In [60]:
agrupado_canoas = agrupado[agrupado["LOCAL"] == "Canoas (RS)"]

In [62]:
agrupado_canoas_exa = agrupado_canoas[agrupado_canoas["MODALIDADE"] == "EXA"]

In [67]:
agrupado_canoas_exa["PRODUTO"] = 0

for line_num, line in enumerate(agrupado_canoas_exa["DATA"]):
    data_corrente = line
    try:
        data_proxima = agrupado_canoas_exa.iloc[line_num + 1, 2]
        if data_corrente < data_proxima:
            agrupado_canoas_exa.iloc[line_num, 4] = "S500"
    except:
        continue

<ipython-input-67-d957a72f3cb9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agrupado_canoas_exa["PRODUTO"] = 0
C:\Users\matheus.vizzotto\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [68]:
print(agrupado_canoas_exa.to_string())

            LOCAL MODALIDADE       DATA   VALOR PRODUTO
18    Canoas (RS)        EXA 2019-08-01  2.0596    S500
94    Canoas (RS)        EXA 2019-09-05  2.1121    S500
170   Canoas (RS)        EXA 2019-09-13  2.1663    S500
246   Canoas (RS)        EXA 2019-09-19  2.2579    S500
322   Canoas (RS)        EXA 2019-10-25  2.2231    S500
398   Canoas (RS)        EXA 2019-11-01  2.1553    S500
474   Canoas (RS)        EXA 2019-11-19  2.1814    S500
550   Canoas (RS)        EXA 2019-12-01  2.1814    S500
626   Canoas (RS)        EXA 2019-12-04  2.2248    S500
702   Canoas (RS)        EXA 2019-12-21  2.3012    S500
778   Canoas (RS)        EXA 2020-01-14  2.2511    S500
854   Canoas (RS)        EXA 2020-01-24  2.1594    S500
930   Canoas (RS)        EXA 2020-01-31  2.0939    S500
1006  Canoas (RS)        EXA 2020-02-06  2.0022    S500
1082  Canoas (RS)        EXA 2020-02-29  1.9017    S500
1158  Canoas (RS)        EXA 2020-03-01  1.9017    S500
18    Canoas (RS)        EXA 2020-03-13  1.7767 

In [54]:
print(agrupado_canoas.reset_index()[agrupado_canoas.reset_index()["PRODUTO"] == "S500"].to_string())

Empty DataFrame
Columns: [index, MODALIDADE, DATA, VALOR, PRODUTO]
Index: []
